In [2]:
!pip install wget

  Using cached wget-3.2.zip (10 kB)
Using legacy 'setup.py install' for wget, since package 'wheel' is not installed.
    Running setup.py install for wget: started
    Running setup.py install for wget: finished with status 'done'


You should consider upgrading via the 'C:\Users\91773\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
!wget https://raw.githubusercontent.com/autosoft-dev/ml-on-code/main/assets/model.py
!wget https://raw.githubusercontent.com/autosoft-dev/ml-on-code/main/assets/utils.py

--2023-04-05 11:43:13--  https://raw.githubusercontent.com/autosoft-dev/ml-on-code/main/assets/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9935 (9.7K) [text/plain]
Saving to: 'model.py'

     0K .........                                             100% 1.50M=0.006s

2023-04-05 11:43:13 (1.50 MB/s) - 'model.py' saved [9935/9935]

--2023-04-05 11:43:14--  https://raw.githubusercontent.com/autosoft-dev/ml-on-code/main/assets/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2602 (2

In [2]:
!pip install transformers
!pip install -U tree-hugger PyYAML

ERROR: huggingface-hub 0.13.3 has requirement packaging>=20.9, but you'll have packaging 20.4 which is incompatible.



  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.2
    Uninstalling typing-extensions-3.7.4.2:
      Successfully uninstalled typing-extensions-3.7.4.2

ERROR: huggingface-hub 0.13.3 has requirement packaging>=20.9, but you'll have packaging 20.4 which is incompatible.
ERROR: Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for tree-sitter: filename=tree_sitter-0.20.1-cp38-cp38-win_amd64.whl size=92693 sha256=871c0d0d74f7238e39829191e4276dd5f68e72627e400d421de962a043d46364
  Stored in directory: c:\users\91773\appdata\local\pip\cache\wheels\fc\9f\3a\3a3803563accdae774c03d8d076d0f80ddcf0698bad61f010a


Successfully built tree-sitter
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.3.1


In [1]:
!pip install torch

In [3]:
import os
import json
import torch
import torch.nn as nn
from model import Seq2Seq
from utils import Example, convert_examples_to_features
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

In [11]:
!wget https://code-summary.s3.amazonaws.com/pytorch_model.bin

--2023-04-05 11:55:37--  https://code-summary.s3.amazonaws.com/pytorch_model.bin
Resolving code-summary.s3.amazonaws.com (code-summary.s3.amazonaws.com)... 52.217.73.28, 52.217.111.116, 54.231.199.177, ...
Connecting to code-summary.s3.amazonaws.com (code-summary.s3.amazonaws.com)|52.217.73.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706871064 (674M) [application/macbinary]
Saving to: 'pytorch_model.bin'

     0K .......... .......... .......... .......... ..........  0%  180K 63m57s
    50K .......... .......... .......... .......... ..........  0%  218K 58m19s
   100K .......... .......... .......... .......... ..........  0% 4.98M 39m38s
   150K .......... .......... .......... .......... ..........  0%  195K 44m27s
   200K .......... .......... .......... .......... ..........  0% 4.17M 36m5s
   250K .......... .......... .......... .......... ..........  0% 1.90M 31m3s
   300K .......... .......... .......... .......... ..........  0% 2.81M 27m11s


^C


In [5]:
## We are defining all the needed functions here. 
def inference(data, model, tokenizer):
    # Calculate bleu
    eval_sampler = SequentialSampler(data)
    eval_dataloader = DataLoader(data, sampler=eval_sampler, batch_size=len(data))

    model.eval()
    p = []
    for batch in eval_dataloader:
        batch = tuple(t.to('cpu') for t in batch)
        source_ids, source_mask = batch
        with torch.no_grad():
            preds = model(source_ids=source_ids, source_mask=source_mask)
            for pred in preds:
                t = pred[0].cpu().numpy()
                t = list(t)
                if 0 in t:
                    t = t[: t.index(0)]
                text = tokenizer.decode(t, clean_up_tokenization_spaces=False)
                p.append(text)
    return (p, source_ids.shape[-1])


def get_features(examples, tokenizer):
    features = convert_examples_to_features(
        examples, tokenizer, stage="test"
    )
    all_source_ids = torch.tensor(
        [f.source_ids[: 256] for f in features], dtype=torch.long
    )
    all_source_mask = torch.tensor(
        [f.source_mask[: 256] for f in features], dtype=torch.long
    )
    return TensorDataset(all_source_ids, all_source_mask)


def build_model(model_class, config, tokenizer):
    encoder = model_class(config=config)
    decoder_layer = nn.TransformerDecoderLayer(
        d_model=config.hidden_size, nhead=config.num_attention_heads
    )
    decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
    model = Seq2Seq(
        encoder=encoder,
        decoder=decoder,
        config=config,
        beam_size=10,
        max_length=128,
        sos_id=tokenizer.cls_token_id,
        eos_id=tokenizer.sep_token_id,
    )

    model.load_state_dict(
        torch.load(
            "pytorch_model.bin",
            map_location=torch.device("cpu"),
        ),
        strict=False,
    )
    return model

In [6]:
config = RobertaConfig.from_pretrained("microsoft/codebert-base")
tokenizer = RobertaTokenizer.from_pretrained(
    "microsoft/codebert-base", do_lower_case=False
)

model = build_model(
    model_class=RobertaModel, config=config, tokenizer=tokenizer
).to('cpu')

In [7]:
example = [Example(source="def add_tensors(t, t1) -> Any:\n    return t + t1", target=None)]
message, length = inference(get_features(example, tokenizer), model, tokenizer)
print(message)

['Add two tensors .']


In [15]:
from pathlib import Path

def check_out_path(target_path: Path):
    """"
    This function recursively yields all contents of a pathlib.Path object
    """
    yield target_path
    for file in target_path.iterdir():
        if file.is_dir():
            yield from check_out_path(file)
        else:
            yield file.absolute()


def is_python_file(file_path: Path):
  """
  This little function will help us to filter the result and keep only the python files
  """
  return file_path.is_file() and file_path.suffix == ".py"

In [17]:
!pip install tree_hugger

  Using cached tree_hugger-0.10.1-py3-none-any.whl (24 kB)
  Using cached pygit2-1.12.0-cp38-cp38-win_amd64.whl (1.2 MB)
Processing c:\users\91773\appdata\local\pip\cache\wheels\fc\9f\3a\3a3803563accdae774c03d8d076d0f80ddcf0698bad61f010a\tree_sitter-0.20.1-cp38-cp38-win_amd64.whl


In [18]:
# We first create our PythonParser object
from tree_hugger.core import PythonParser

In [31]:
pp = PythonParser(library_loc="C:/Nish/CodeSummary UI/my-languages.so")

In [33]:
for file_path in check_out_path(Path("training-file-fyp")):
    if is_python_file(file_path):
      # we use one line, super convinient tree-hugger API call to get the needed data
      if pp.parse_file(str(file_path)):
        temp_cache = []
        # The following call returns a dict where each key is a name of a function
        # And each value is a tuple, (function_body, function_docstring)
        func_and_docstr = pp.get_all_function_bodies(strip_docstr=True)
        for func_name, (body, docstr) in func_and_docstr.items():
          example = [Example(source=body, target=None)]
          message, length = inference(get_features(example, tokenizer), model, tokenizer)
          print(message)

['Perform a HTTP request .']
['Send a GET request .']


KeyboardInterrupt: 